### Project: RDKit - 3Dmol.js integration<br>Function : MolView3D and a few utilities <br><br>Notebook 3 : Visualizing energy minimization effect on ligand multiple conformers
Student: Malitha Humayun Kabir<br>
Mentors: Paul Czodrowski and Greg Landrum <br>
Date: 12th September 2017 <br>
Acknowledgement: Peter Gedeck reviewed and provided advice on restructuring codes and wrote MolViewState class. The starting point of MolView3D was Greg Landrum's codes on conformer browser.

In [ ]:
from rdkit.Chem.Draw import IPythonConsoleIntegration as ipy
from rdkit.Chem.ChemUtils import LigandExtract
from rdkit import Chem
from rdkit.Chem import AllChem
from six.moves import urllib

This notebook demonstrates rapid <b>extration of ligand from PDB file, energy minimization of ligand</b>, and subsequent <b>visualization of protein-ligand complex</b> using <b>MolView3D</b> function of RDKit. This notebook extends the information presented at <b>notebook 2</b>.

Let's start from ligand extraction. We can use any PDB file. 

In [ ]:
url = 'https://files.rcsb.org/download/4ICZ.pdb'
response = urllib.request.urlopen(url)
data = response.read()

Variable name <b>data</b> got the required data that can be used in creating RDKit Mol object.

In [ ]:
Mol = Chem.MolFromPDBBlock(data)
Mol is None

The residue <b>PTR</b> is the ligand. We are now extracting it.

In [ ]:
nm = LigandExtract.ExtractMolFragment(Mol,ResName = 'PTR')
nm

Variable <b>nm</b> contains two elements: <ul><li><b>nm.match</b> is the ligand </li><li> <b>nm.rest</b> is the protein.</li></ul>

Now we need to put the ligand in a dictionary.

In [ ]:
moldict = {'0': nm.match}
moldict

Running the <b>ProcessLigandDict</b> function for the dictionary <b>moldict</b> does the required processing generating another dictionary. The later one is in suitable format for <b>MolView3D</b> function.

In [ ]:
moldict_2 = ipy.ProcessLigandDict(moldict)
moldict_2

As we see here, <b>ProcessLigandDict</b> actually adds another dictionary under each <b>molecule id</b> while the original mol object taken to the new dictionary. For the convenience of the presentation, we call this assigned key for the new dictionary is <b>child key</b>. 

In [ ]:
ipy.CopyMolWithChildKey(ligandDict = moldict_2, copyFrom = 'lig', newChildKey = 'emLig')
moldict_2

The function - <b>CopyMolWithChildKey</b> adds another dictionary under each <b>molecule id</b>. We can also choose the key name as per our wish. Before running the function <b>CopyMolWithChildKey</b> the child key <b>lig</b> was in <b>moldict_2</b>. So, the <b>copyFrom</b> argument is <b>lig</b>, and the function adds another dictionary with the key <b>emLig</b> under molecule id. So, <b>newChildKey</b> argument is <b>emLig</b>. 

In [ ]:
steps = 4
iterPerStep = 50
mol = moldict_2['0']['emLig']

for i in range(steps):
    ipy.MinimizeLigand(ligandDict = moldict_2, 
                       onChildKey = 'emLig', createPerConfDataKey = 'energy',
                       molAndConfIds = (('0', i),), 
                       maxIters = iterPerStep)
    if mol.GetNumConformers() < steps:
        mol.AddConformer(mol.GetConformer(i) ,assignId=True)
        
moldict_2

In the above code block, energy minimization performed on the molecule with child key <b>emLig</b>. On each iteration, the energy minimized conformer serves as the starting conformer of the next iteration. And the minimization energy added to the dictionary with key <b>energy</b> under molecule id.

As we see in <b>moldict_2</b>, the molecule having child key <b>lig</b> got only one conformer and <b>emLig</b> got more than one conformer, we need to ask <b>MolView3D</b> to take conformer id from <b>emLig</b> for conformer selection and also where the minimization energy are stored. Another thing is, if we want to view energy level data in <b>MolView3D widgets</b>, then we also need to parse another argument <b>emPanel = True</b>.

Let's view this in <b>MolView3D</b>. Please put tick mark at checkbox named <b>selectAllConfs</b>. This will show all the energy minimized conformers.

In [ ]:
view = ipy.MolView3D(ligandDict = moldict_2, protein = nm.rest, 
                     childKeysForMolRender = ['lig', 'emLig'], 
                     childKeyForConfSelection = 'emLig',
                     childKeyForDataPerConf = 'energy',
                     emPanel = True,
                     ligStyle = ['stick', 'stick'], protStyle='cartoon',
                     ligColor = ['default','orangeCarbon'], protColor='ssPyMOL')

Please also note that, at the time of writing this notebook (11th September 2017), <b>surface coloring for both protein and ligand will not work</b>. We hope to add this feature in near future.

In [ ]:
view2 = ipy.MolView3D(ligandDict = moldict_2, protein = nm.rest, 
                      childKeysForMolRender = ['lig'], 
                      childKeyForConfSelection = 'lig',
                      ligStyle = ['stick'], protStyle='surface',
                      ligColor = ['default'])

### Thank you very much for your time. Have a great day! 